In [1]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import animation
import h5py
import os, sys
from voxelgrid import VoxelGrid
from plot3D import *

%matplotlib inline

/home/isaac/miniconda3/envs/3dvision/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plt.rcParams['image.cmap'] = 'gray'

with h5py.File('./data/train_point_clouds.h5', 'r') as f:
    # Reading digit at zeroth index    
    a = f[str(1)]   
    # Storing group contents of digit a
    digit = (a["img"][:], a["points"][:], a.attrs["label"])

In [3]:
# data = np.empty((500, 2048, 3))
# label = np.empty((500))
# with h5py.File('./data/train_point_clouds.h5') as f:
#     for i in range(500):
#         d = f[str(i)]
#         idxs = np.arange(0, d["points"][:].shape[0])
#         np.random.shuffle(idxs)
#         data[i,:,:] = d["points"][:][idxs[:2048]]
#         label[i] = d.attrs["label"]


In [4]:
digit[2], digit[1].shape

(0, (26200, 3))

In [5]:
def vis_3d(digit):
    %matplotlib

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    X = digit[1][:,0]
    Y = digit[1][:,1]
    Z = digit[1][:,2]
    ax.scatter(X,Y,Z)


In [6]:
vis_3d(digit)

Using matplotlib backend: TkAgg


In [7]:
voxel_grid = VoxelGrid(digit[1], x_y_z = [16, 16, 16])

digits = []
with h5py.File("./data/train_point_clouds.h5", 'r') as h5:
    for i in range(15):
        d = h5[str(i)]
        digits.append((d["img"][:],d["points"][:],d.attrs["label"]))
        
def count_plot(array_):
    cm = plt.cm.get_cmap('gist_rainbow')
    n, bins, patches = plt.hist(array_, bins=64)

    bin_centers = 0.5 * (bins[:-1] + bins[1:])

    # scale values to interval [0,1]
    col = bin_centers - min(bin_centers)
    col /= max(col)

    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    plt.show()
   
# Get the count of points within each voxel.
plt.title("DIGIT: " + str(digits[0][-1]))
plt.xlabel("VOXEL")
plt.ylabel("POINTS INSIDE THE VOXEL")
count_plot(voxel_grid.structure[:,-1])    


voxels = []
for d in digits:
    voxels.append(VoxelGrid(d[1], x_y_z=[16,16,16]))
    
# Visualizing the Voxel Grid sliced around the z-axis.
voxels[0].plot()
plt.show()

## PointNet

In [6]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket
import importlib
import os
import sys

In [8]:
BASE_DIR = os.path.dirname(os.getcwd())
BASE_DIR = os.path.join(BASE_DIR, '3d-mnist')
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
import provider
import tf_util

In [9]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--gpu', type=int, default=0, help='GPU to use [default: GPU 0]')
# parser.add_argument('--model', default='pointnet_cls', help='Model name: pointnet_cls or pointnet_cls_basic [default: pointnet_cls]')
# parser.add_argument('--log_dir', default='log', help='Log dir [default: log]')
# parser.add_argument('--num_point', type=int, default=1024, help='Point Number [256/512/1024/2048] [default: 1024]')
# parser.add_argument('--max_epoch', type=int, default=250, help='Epoch to run [default: 250]')
# parser.add_argument('--batch_size', type=int, default=32, help='Batch Size during training [default: 32]')
# parser.add_argument('--learning_rate', type=float, default=0.001, help='Initial learning rate [default: 0.001]')
# parser.add_argument('--momentum', type=float, default=0.9, help='Initial learning rate [default: 0.9]')
# parser.add_argument('--optimizer', default='adam', help='adam or momentum [default: adam]')
# parser.add_argument('--decay_step', type=int, default=200000, help='Decay step for lr decay [default: 200000]')
# parser.add_argument('--decay_rate', type=float, default=0.7, help='Decay rate for lr decay [default: 0.8]')
# FLAGS = parser.parse_args()

In [10]:
BATCH_SIZE = 32
NUM_POINT = 1024
MAX_EPOCH = 250
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 200000
DECAY_RATE = 0.7

# MODEL = importlib.import_module('pointnet_cls') # import network module
from models import pointnet_cls as MODEL

MODEL_FILE = os.path.join(BASE_DIR, 'models', 'pointnet_cls.py')
LOG_DIR = 'log'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp %s %s' % (MODEL_FILE, LOG_DIR)) # bkp of model def
os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

MAX_NUM_POINT = 2048
NUM_CLASSES = 40

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()


In [13]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def train():
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())
            print(is_training_pl)
            
            # Note the global_step=batch parameter to minimize. 
            # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
            batch = tf.Variable(0)
            bn_decay = get_bn_decay(batch)
            tf.summary.scalar('bn_decay', bn_decay)

            # Get model and loss 
            pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
            loss = MODEL.get_loss(pred, labels_pl, end_points)
            tf.summary.scalar('loss', loss)

            correct = tf.equal(tf.argmax(pred, 1), tf.to_int64(labels_pl))
            accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE)
            tf.summary.scalar('accuracy', accuracy)

            # Get training operator
            learning_rate = get_learning_rate(batch)
            tf.summary.scalar('learning_rate', learning_rate)
            if OPTIMIZER == 'momentum':
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
            elif OPTIMIZER == 'adam':
                optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.minimize(loss, global_step=batch)
            
            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()
            
        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = False
        sess = tf.Session(config=config)

        # Add summary writers
        #merged = tf.merge_all_summaries()
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

        # Init variables
        init = tf.global_variables_initializer()
        # To fix the bug introduced in TF 0.12.1 as in
        # http://stackoverflow.com/questions/41543774/invalidargumenterror-for-tensor-bool-tensorflow-0-12-1
        #sess.run(init)
        sess.run(init, {is_training_pl: True})

        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'pred': pred,
               'loss': loss,
               'train_op': train_op,
               'merged': merged,
               'step': batch}

        for epoch in range(MAX_EPOCH):
            log_string('**** EPOCH %03d ****' % (epoch))
            sys.stdout.flush()
             
            train_one_epoch(sess, ops, train_writer)
            eval_one_epoch(sess, ops, test_writer)
            
            # Save the variables to disk.
            if epoch % 10 == 0:
                save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
                log_string("Model saved in file: %s" % save_path)



def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('---------')
    current_data, current_label = provider.loadDataFile("./data/train_point_clouds.h5", 'train')
#     print(current_data.shape, current_label.shape)
    current_data = current_data[:,0:NUM_POINT,:]
    current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))            
    current_label = np.squeeze(current_label)

    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE

    total_correct = 0
    total_seen = 0
    loss_sum = 0

    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        # Augment batched point clouds by rotation and jittering
        rotated_data = provider.rotate_point_cloud(current_data[start_idx:end_idx, :, :])
        jittered_data = provider.jitter_point_cloud(rotated_data)
        feed_dict = {ops['pointclouds_pl']: jittered_data,
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'],
            ops['train_op'], ops['loss'], ops['pred']], feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 1)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += BATCH_SIZE
        loss_sum += loss_val
        
        log_string('mean loss: %f' % (loss_sum / float(num_batches)))
        log_string('accuracy: %f' % (total_correct / float(total_seen)))

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    
    log_string('---------')
    current_data, current_label = provider.loadDataFile("./data/test_point_clouds.h5", 'test')
    current_data = current_data[:,0:NUM_POINT,:]
    current_label = np.squeeze(current_label)

    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE

    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}
        summary, step, loss_val, pred_val = sess.run([ops['merged'], ops['step'],
            ops['loss'], ops['pred']], feed_dict=feed_dict)
        pred_val = np.argmax(pred_val, 1)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += BATCH_SIZE
        loss_sum += (loss_val*BATCH_SIZE)
        for i in range(start_idx, end_idx):
            l = int(current_label[i])
            total_seen_class[l] += 1
            total_correct_class[l] += (pred_val[i-start_idx] == l)
            
    log_string('eval mean loss: %f' % (loss_sum / float(total_seen)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
         

In [ ]:
train()
LOG_FOUT.close()

Tensor("Placeholder_2:0", shape=(), dtype=bool, device=/device:GPU:0)
INFO:tensorflow:Summary name classify loss is illegal; using classify_loss instead.
INFO:tensorflow:Summary name mat loss is illegal; using mat_loss instead.
**** EPOCH 000 ****
---------
mean loss: 0.017768
accuracy: 0.125000
mean loss: 0.040167
accuracy: 0.125000
mean loss: 0.065763
accuracy: 0.135417
mean loss: 0.095470
accuracy: 0.156250
mean loss: 0.123953
accuracy: 0.193750
mean loss: 0.150863
accuracy: 0.187500
mean loss: 0.175069
accuracy: 0.187500
mean loss: 0.195374
accuracy: 0.191406
mean loss: 0.215360
accuracy: 0.194444
mean loss: 0.236125
accuracy: 0.190625
mean loss: 0.255379
accuracy: 0.190341
mean loss: 0.273695
accuracy: 0.197917
mean loss: 0.294123
accuracy: 0.194712
mean loss: 0.314542
accuracy: 0.189732
mean loss: 0.333873
accuracy: 0.195833
mean loss: 0.354361
accuracy: 0.197266
mean loss: 0.375384
accuracy: 0.198529
mean loss: 0.393947
accuracy: 0.199653
mean loss: 0.412449
accuracy: 0.208882
m

/home/isaac/miniconda3/envs/3dvision/lib/python3.6/site-packages/ipykernel_launcher.py:185: RuntimeWarning: invalid value encountered in true_divide


Model saved in file: log/model.ckpt
**** EPOCH 001 ****
---------
mean loss: 0.010468
accuracy: 0.437500
mean loss: 0.021212
accuracy: 0.468750
mean loss: 0.031374
accuracy: 0.479167
mean loss: 0.042934
accuracy: 0.468750
mean loss: 0.056784
accuracy: 0.437500
mean loss: 0.065918
accuracy: 0.458333
mean loss: 0.076910
accuracy: 0.468750
mean loss: 0.085742
accuracy: 0.472656
mean loss: 0.097759
accuracy: 0.458333
mean loss: 0.108587
accuracy: 0.462500
mean loss: 0.119383
accuracy: 0.465909
mean loss: 0.128847
accuracy: 0.468750
mean loss: 0.140137
accuracy: 0.471154
mean loss: 0.149543
accuracy: 0.470982
mean loss: 0.157898
accuracy: 0.481250
mean loss: 0.171836
accuracy: 0.466797
mean loss: 0.186647
accuracy: 0.459559
mean loss: 0.198322
accuracy: 0.456597
mean loss: 0.209886
accuracy: 0.452303
mean loss: 0.220454
accuracy: 0.457813
mean loss: 0.229963
accuracy: 0.458333
mean loss: 0.241445
accuracy: 0.461648
mean loss: 0.251667
accuracy: 0.464674
mean loss: 0.263113
accuracy: 0.46224